In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [2]:
# Data Directory
data_dir = 'dataset/'

# Load the data
train_1 = pd.read_csv(data_dir + 'train_1.csv')
train_2_1 = pd.read_csv(data_dir + 'train_2_1.csv')
train_2_2 = pd.read_csv(data_dir + 'train_2_2.csv')
test_1 = pd.read_csv(data_dir + 'test_1.csv')
test_2_1 = pd.read_csv(data_dir + 'test_2_1.csv')
test_2_2 = pd.read_csv(data_dir + 'test_2_2.csv')

/tmp/ipykernel_15752/3751997310.py:5: DtypeWarning: Columns (126,128,143) have mixed types. Specify dtype option on import or set low_memory=False.
  train_1 = pd.read_csv(data_dir + 'train_1.csv')
/tmp/ipykernel_15752/3751997310.py:6: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train_2_1 = pd.read_csv(data_dir + 'train_2_1.csv')
/tmp/ipykernel_15752/3751997310.py:7: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train_2_2 = pd.read_csv(data_dir + 'train_2_2.csv')


In [3]:
# Merge datasets
train = pd.merge(train_1, train_2_1, on='id', how='left')
train = pd.merge(train, train_2_2, on='id', how='left')

test = pd.merge(test_1, test_2_1, on='id', how='left')
test = pd.merge(test, test_2_2, on='id', how='left')


In [4]:
from scipy import stats

# Identify numerical and categorical columns
numerical_cols_train = train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_train = train.select_dtypes(include=['object']).columns

numerical_cols_test = test.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_test = test.select_dtypes(include=['object']).columns

# Handle missing values for numerical columns train
for col in numerical_cols_train:
    train[col] = train[col].fillna(train[col].mean())
# Handle missing values for categorical columns train
for col in categorical_cols_train:
    train[col] = train[col].fillna('missing')

# Handle missing values for numerical columns test
for col in numerical_cols_test:
    test[col] = test[col].fillna(test[col].mean())
# Handle missing values for categorical columns test
for col in categorical_cols_test:
    test[col] = test[col].fillna('missing')

# # Handle outliers for numerical columns using Z-score
# for col in numerical_cols_train:
#     train = train[(np.abs(stats.zscore(train[col])) < 3)]
# for col in numerical_cols_test:
#     test = test[(np.abs(stats.zscore(test[col])) < 3)]

In [5]:
# Select relevant features
features = train.columns.drop(['loan_id', 'id', 'label'])
X_train = train[features]
y_train = train['label']
X_test = test[features]

In [6]:
# Remove 'loan_id' and 'id' from the list of categorical columns if they exist
categorical_cols_train = [col for col in categorical_cols_train if col not in ['loan_id', 'id']]
categorical_cols_test = [col for col in categorical_cols_test if col not in ['loan_id', 'id']]

# Remove 'loan_id' and 'id' from the features if they exist
X_train = X_train.drop(columns=['loan_id', 'id'], errors='ignore')
X_test = X_test.drop(columns=['loan_id', 'id'], errors='ignore')

# Convert categorical columns to numerical using one-hot encoding
X_train = pd.get_dummies(X_train, columns=categorical_cols_train)
X_test = pd.get_dummies(X_test, columns=categorical_cols_test)

# Ensure the same columns in train and test after one-hot encoding
X_train, X_test = X_train.align(X_test, join='inner', axis=1, fill_value=0)

: 